<a href="https://colab.research.google.com/github/heyheydesu/amazon-codeguru-reviewer-sample-app/blob/master/02_netapp_createConfigSheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
#@title NetAppの設定書を自動的につくる

# 使い方
# 01_nettapp_getParamater.txtを機器で実行し、パラメータを取得する。
# http://400cc.blog.fc2.com/blog-entry-113.html

# 出力された結果をテキストに貼りつけ、保存する。

# テキストを左のファイルにドラッグアンドドロップする

# ファイル名をnetappp_souce_fileに入力する。

# 実行　( ctlr + F9 )

#[課題]  ,カンマで改行したい

#[課題]  snmp-traphostが表にならない



In [97]:
#@title ソースファイル名を入力
netapp_source_file = 'sample.txt' #@param {type:"string"}
netapp_source_path = '/content/' + netapp_source_file

anken_name = "test" #@param {type:"string"}
customer_corp = "test" #@param {type:"string"}
customer_name = "test"  #@param {type:"string"}
customer_tel = "test" #@param {type:"string"}
customer_mail = "test" #@param {type:"string"}
location_address= "test"#@param {type:"string"}


customer_list = [
                 ['会社名' , customer_corp],
                 ['担当者' , customer_name],
                 ['電話番号' , customer_tel],
                 ['メールアドレス' , customer_mail],
                 ['設置先情報' , location_address],
                 ]


In [98]:
#@title 検索文字列リスト

cluster_word_list =  ["(Cluster Name):(.*)" ,"cluster_name"]

# 置換するワード(完全一致)
replace_word_list = [
                      ['#_ ',""],
                      ["sheet.",""],
                      ["Sheet",""],
                      ['version -node *',""],
                      ['NetApp Release ',""],
                      ['Cluster Location: \n',""],
                      ['Cluster Contact: \n',""],
                      ['(from "system snmp traphost")',""],
                      ["There are no entries matching your query." , "設定無し"],
                      [ '-spare-disks',""],
                      ['original-owner',"node"],

                   ]

# 置換するワード（正規表現）
sub_word_list = [
                  ['Cluster UUID: .{2,300}\n',""],
                  ['Cluster Name: .{2,300}\n',""],
                  ['Cluster Serial Number: .{2,300}\n',""],
                  ['\d{1,2} entries were displayed.',""],
                  ['\n-*-',""],
                  [ '--*--',""],
                  ['.{2,100} show|-fields.{2,300}\n',""],
                  ['local-usable-root-size',"root-partiton"],
                  ['local-usable-data-size',"data-patition"],
                ]



In [99]:
#@title [import] ライブラリ読み込み

#エクセル操作
!pip install openpyxl

#ライブラリのインポート
import re

#エクセル操作
import openpyxl
import pprint
import glob
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment

#フォント
from openpyxl.styles.fonts import Font

#ファイル操作用
import shutil
import os

# 時間取得
import datetime

# ファイルダウンロード
from google.colab import files

In [100]:
#@title　 [関数]  ファイル、フォルダ初期化

def initConfigfolder():

  #configフォルダがあれば、削除
  if os.path.exists('/content/config'):
    shutil.rmtree('/content/config')

  #configフォルダを新規作成
  os.mkdir('/content/config')

In [101]:
#@title [関数]  元テキストデータを読み込み
def readConfig():
  config_path = netapp_source_path
  config_raw = open( config_path, 'r' ,encoding="ISO-8859-1")
  config_text = config_raw.read()

  return config_text

In [102]:
#@title [関数]　検索する文字の行を取得する関数
def getPatternline( word_line):

  #検索文字列を代入
  pattern = word_line[0]

  #検索して行を抽出
  res = re.findall(pattern, readConfig())
  print (res)
  return res

In [103]:
#@title [関数] 不要な文字列を削除

def replaceConfig( ):

  # <先頭文字列>を削除
  config_text001 = readConfig().replace( cluster_name + '::*>' , "")
  config_text001 = config_text001.replace( cluster_name + '::>' , "")

  for replace in range (len(sub_word_list)) :
    # 不要文字リストを削除
    config_text001 = re.sub( sub_word_list[replace][0] , sub_word_list[replace][1]  , config_text001 )

  for delete in range (len(replace_word_list)) :
    # 不要文字リストを削除
    config_text001 = config_text001.replace( replace_word_list[delete][0]  , replace_word_list[delete][1])


  return config_text001

In [104]:
#@title [関数]　元テキストを分割
def splitConfig(  ):

  # 元テキストデータを分割し、1次元リストに代入
  config_split_list = replaceConfig().split( '#___' )

  # テキストファイルを分割（リストごとにテキストファイルを作成）
  for line in range ( len(config_split_list) ):

    #ファイルを新規作成
    config_split_path =  '/content/config/config_' + str(line).zfill(3) + '.txt'
    f = open(config_split_path , 'w' )

    #ファイルを書き込み
    f.write( config_split_list[ line ])
    f.close

  return config_split_list

In [105]:
#@title [関数] 分割したテキストファイルの読み込み

def readConfigsplit( config_file ):
  config_path = config_file
  config_raw = open(config_path , 'r' ,encoding="UTF-8")

  # 1行ずつ配列に追加
  row_no = 0
  config_rows = []

  while True :
    config_rows.append( config_raw.readline() )

    if config_rows[row_no]:
      row_no += 1
    else :
      break

  # 1行ずつ配列に追加
  config_list = []
  for config_i in range ( len(config_rows) ):
    config_list.append( config_rows[ config_i ].split() )

  return config_list

In [106]:
#@title [関数] 印刷設定
def printConfig(current_sheet):
  current_sheet.page_setup.fitToWidth = 1
  current_sheet.page_setup.fitToHeight = 0
  current_sheet.sheet_properties.pageSetUpPr.fitToPage = True


In [107]:
#@title [関数] タイトル装飾
def setSheettitle (current_sheet):

  #タイトル文字色を白に変更
  current_sheet['B2'].font = Font(color='FFFFFF' ,size=18)

  #タイトル行を紺色にする
  for color_rows in current_sheet['B2':'I2']:
    for cell in color_rows:
      cell.fill = openpyxl.styles.PatternFill(patternType='solid',fgColor='191970',bgColor='191970')


  #2行目の列幅を広くする
  current_sheet.row_dimensions[2].height = 32



In [108]:
#@title [関数] テーブルの描画
def drawTable( current_sheet ,table_start ,table_end, table_count): # Add table_count parameter):

  # テーブルスタイルを定義
  tableStyle = openpyxl.worksheet.table.TableStyleInfo( name='TableStyleMedium2',  showRowStripes=True)

  # テーブルの範囲を定義
  # table = openpyxl.worksheet.table.Table( ref= table_start +':' + table_end  ,displayName='',tableStyleInfo=tableStyle, headerRowCount=1)
  # table_count を使用して一意の表示名を割り当てる
  table_name = f'Table_{current_sheet.title}_{table_count}'
  table = openpyxl.worksheet.table.Table(
      ref= table_start +':' + table_end ,
      displayName=table_name,  # 一意のテーブル名を使用
      tableStyleInfo=tableStyle,
      headerRowCount=1)

  #フィルタが消える　（しかしヘッダの書式も消える）
  #table = openpyxl.worksheet.table.Table( ref= table_start +':' + table_end  ,displayName='',tableStyleInfo=tableStyle, headerRowCount=0)

  #テーブルを挿入
  current_sheet.add_table(table)



In [109]:
#@title [関数] A列を狭く、B～H列の幅調整
def setColume_size (current_sheet):
  current_sheet.column_dimensions['A'].width = 3
  for i in range(ord('B'),ord('I')+1):
    current_sheet.column_dimensions[chr(i)].width = 15


In [110]:
#@title [関数] 表紙作成
def createCover(cluster_name):

  book.create_sheet( '表紙' )
  current_sheet = book[ '表紙']

  #目盛り線削除
  current_sheet.sheet_view.showGridLines = False

  #枠作成（セル結合）
  current_sheet.merge_cells( 'B34:H38' )
  current_sheet.merge_cells( 'B39:H41' )
  current_sheet.merge_cells( 'B42:H45' )
  current_sheet.merge_cells( 'D46:F48' )

  #列幅
  current_sheet.column_dimensions['A'].width = 3
  current_sheet.column_dimensions['I'].width = 3

  #案件名
  current_sheet['H32'].value = '1.0.0版'
  current_sheet['H32'].alignment = Alignment(horizontal = 'right', vertical = 'center', wrap_text = False)

  #案件名
  current_sheet['B34'].value = anken_name
  current_sheet['B34'].font  = Font(size=16 )
  current_sheet['B34'].alignment = Alignment(horizontal = 'center', vertical = 'center', wrap_text = False)

  #機器、OS名
  current_sheet['B39'].value = '設定シート'
  current_sheet['B39'].font  = Font(size=26 )
  current_sheet['B39'].alignment = Alignment(horizontal = 'center', vertical = 'center', wrap_text = False)

  #機器、OS名
  current_sheet['B42'].value = 'NetApp Clustered ONTAP 9'
  current_sheet['B42'].font  = Font(size=26 )
  current_sheet['B42'].alignment = Alignment(horizontal = 'center', vertical = 'center', wrap_text = False)


  #クラスター名
  current_sheet['D46'].value = 'クラスター名 : ' + cluster_name
  current_sheet['D46'].font  = Font(size=11 )
  current_sheet['D46'].alignment = Alignment(horizontal = 'center', vertical = 'center', wrap_text = False)


  # 罫線スタイル
  side = Side(style='thin', color='000000')
  border = Border(top=side, bottom=side, left=side, right=side)

  #罫線を描画
  for rows in current_sheet['B34':'H38']:
    for cell in rows:
        cell.border = Border(left = side, right = side, top = side, bottom = side)

  #列幅を調整
  for i in range(ord('B'),ord('H')+1):
    current_sheet.column_dimensions[chr(i)].width = 15

  #印刷設定
  printConfig(current_sheet)

In [111]:
#@title [関数] 改訂履歴シート作成
def createHistory():
  #　改訂履歴シートを作成----------------------------------
  book.create_sheet( '改訂履歴' )
  current_sheet = book[ '改訂履歴']

  #目盛り線削除
  current_sheet.sheet_view.showGridLines = False

  # A列を狭く、B～H列の幅調整
  setColume_size (current_sheet)

  #印刷設定
  printConfig( current_sheet)

  #タイトル行を紺色に変更
  setSheettitle(current_sheet)

  #タイトルを記入
  current_sheet['B2'].value = '改訂履歴'

  # datetimeインスタンスを作成
  dt_now = datetime.datetime.now()

  #列項目を作成
  current_sheet['B4'].value = '●改訂履歴'
  current_sheet['B5'].value = '版数'
  current_sheet['C5'].value = '改訂日'
  current_sheet['D5'].value = '改訂内容'
  current_sheet['I5'].value = '改訂者'

  # 2列目を記入
  current_sheet['B6'].value = '1.0'
  current_sheet['C6'].value = dt_now.strftime('%Y/%m/%d')
  current_sheet['D6'].value = '新規作成'
  current_sheet['I6'].value = 'CTC'


  # 表作成
  for border_y in range (5, 15):
    #セル結合
    current_sheet.merge_cells( current_sheet.cell( row=border_y , column= 4 ).coordinate +':' + current_sheet.cell( row=border_y , column= 8 ).coordinate )

    # 罫線スタイル
    side = Side(style='thin', color='000000')
    border = Border(top=side, bottom=side, left=side, right=side)

    for border_x in range (2, 10):
      #罫線
      current_sheet[ current_sheet.cell( row=border_y , column= border_x ).coordinate ].border = border
      #列名を白に変更
      current_sheet[current_sheet.cell( row=5 , column= border_x ).coordinate].font = Font(color='FFFFFF' )
      #列背景色を白に変更
      current_sheet[current_sheet.cell( row=5 , column= border_x ).coordinate].fill = openpyxl.styles.PatternFill(patternType='solid',fgColor='6495ED',bgColor='6495ED')



In [112]:
#@title [関数] コンフィグシート作成
def createConfigsheet():
  #シートを周回
  for configfile_i in  range ( len(splitConfig()) ):
    config_list = readConfigsplit( '/content/config/config_' + str(configfile_i).zfill(3) + '.txt' )

    # エクセルファイルに書き込み

    #シート名を設定
    sheet_name = 'config_' + str(configfile_i).zfill(3)

    #シートを作成
    book.create_sheet( sheet_name )

    #作成したシートに切り替え
    current_sheet = book[sheet_name]

    #目盛り線削除
    current_sheet.sheet_view.showGridLines = False


    #印刷設定
    printConfig(current_sheet)

    # A列を狭く、B～H列の幅調整
    setColume_size ( current_sheet )

    # セルへの書き込み開始-----------------------------------

    # 1行ずつ読み込み
    for write_y in range (1, len(config_list)) :

      # さらに1セルずつ読み込み
      for write_x in range (1, len(config_list[write_y -1])+1 ):

        #セルに書き込み（スペースでセルを区切る）
        current_sheet.cell( row=write_y+1, column=write_x+1 ).value = config_list[ write_y -1][write_x -1 ]
        cell_now = current_sheet.cell( row=write_y+1, column=write_x+1 )



In [113]:
#@title [関数]テーブル作成
def createTable():
  #シートを周回
  for configfile_i in  range ( len(splitConfig()) ):
    config_list = readConfigsplit( '/content/config/config_' + str(configfile_i).zfill(3) + '.txt' )

    #シート名を設定
    sheet_name = 'config_' + str(configfile_i).zfill(3)

    #シートを切り替え
    current_sheet = book[sheet_name]

    #タイトル行を紺色に変更
    setSheettitle( current_sheet)

    table_count = 0 # テーブルカウントを初期化

    # 1行ずつ読み込み
    for write_y in range (3, len(config_list)) :

      # さらに1セルずつ読み込み
      for write_x in range (1, len(config_list[write_y -1])+1 ):

        #現在のセルの位置を取得
        cell_now = current_sheet.cell( row=write_y+1, column=write_x+1 )

        #現在のセルが空じゃない && ←左のセルが空 && ↑上のセルが空
        #高さがあってない
        if  cell_now.value  != '' and  current_sheet.cell( row=write_y+1, column=write_x ).value is None and current_sheet.cell( row=write_y, column=write_x+1 ).value is None:
          # 現在のセルの真下のセルを取得
          table_start = current_sheet.cell( row=write_y+2, column=write_x+1 ).coordinate

        #現在のセルが空じゃない && ↑上のセルが空じゃない && →右のセルが空 && ↓下のセルが空
        elif  cell_now.value  != '' and current_sheet.cell( row=write_y, column=write_x+1 ).value is not None and current_sheet.cell( row=write_y+1, column=write_x+2 ).value is None and current_sheet.cell( row=write_y+2, column=write_x+1 ).value is None:
          table_end = current_sheet.cell( row=write_y+1 , column= write_x +1 ).coordinate

          #テーブルを作成
          table_count += 1 # テーブルカウントをインクリメント
          drawTable( current_sheet ,table_start ,table_end, table_count )  # table_count を渡す
          # drawTable( current_sheet ,table_start ,table_end )

In [114]:
#@title [関数]シートタイトルの変更
def changeSheetname():
  for configfile_i in  range ( len(splitConfig()) ):
    config_list = readConfigsplit( '/content/config/config_' + str(configfile_i).zfill(3) + '.txt' )

    #シート名を設定
    sheet_name = 'config_' + str(configfile_i).zfill(3)

    #シートタイトルを変更（空のときはスキップ）
    if config_list[0] != [] :
      book[ sheet_name ].title = config_list[0][0]

In [115]:
#@title [関数]設置先情報シート作成
def createCustomer():
  book.create_sheet( '設置先情報' )
  current_sheet = book[ '設置先情報']

  #目盛り線削除
  current_sheet.sheet_view.showGridLines = False

  # A列を狭く、B～H列の幅調整
  setColume_size (current_sheet)

  #印刷設定
  printConfig(current_sheet)

  #タイトル行を紺色に変更
  setSheettitle(current_sheet)

  #タイトルを記入
  current_sheet['B2'].value = '設置先情報'

  # 表作成
  for vborder_y in range ( len(customer_list)*3 ):

    #セル結合
    current_sheet.merge_cells( current_sheet.cell( row=vborder_y +5 , column= 2 ).coordinate +':' + current_sheet.cell( row=vborder_y+7 , column= 3 ).coordinate )
    current_sheet.merge_cells( current_sheet.cell( row=vborder_y +5 , column= 4 ).coordinate +':' + current_sheet.cell( row=vborder_y+7 , column= 8 ).coordinate )

    #列名を白に変更
    current_sheet[current_sheet.cell( row=vborder_y +5 , column= 2 ).coordinate].font = Font(color='FFFFFF' )

    #列背景色を青に変更
    current_sheet[current_sheet.cell( row=vborder_y +5  , column= 2 ).coordinate].fill = openpyxl.styles.PatternFill(patternType='solid',fgColor='6495ED',bgColor='6495ED')

  #罫線を描画
  for rows in current_sheet['B5':'H19']:
    for cell in rows:
        cell.border = Border(left = side, right = side, top = side, bottom = side)


  #列項目を作成
def createCustomer():
  book.create_sheet( '設置先情報' )
  current_sheet = book[ '設置先情報']

  # ... (他のコード) ...

  # 列項目を作成
  write_row = 5  # 初期行を設定
  for customer_y in range(len(customer_list)):
    write_col = 2  # 初期列を設定
    for customer_x in range(len(customer_list[0])):
      # ... (残りのコード) ...

      # ターゲットセルが結合された範囲内にあるかどうかを確認する
      if not isinstance(current_sheet.cell(row=write_row, column=write_col), openpyxl.cell.cell.MergedCell):
        # 結合されていない場合は、値を書き込む
        current_sheet.cell(row=write_row, column=write_col).value = customer_list[customer_y][customer_x]
      else:
        # 結合されている場合は、結合された範囲の左上のセルを見つけて書き込む
        merged_cell_ranges = current_sheet.merged_cells.ranges
        for merged_cell_range in merged_cell_ranges:
          if current_sheet.cell(row=write_row, column=write_col).coordinate in merged_cell_range:
            top_left_cell = current_sheet.cell(row=merged_cell_range.min_row, column=merged_cell_range.min_col)
            top_left_cell.value = customer_list[customer_y][customer_x]
            break  # 見つけて書き込んだので、それ以上のチェックは不要

      write_col += 2  # 次の列へ移動 (2列ずつ)
    write_row += 3  # 次の行へ移動 (3行ずつ)


In [116]:
#@title クラスター名を取得

exec( '{} = {}'.format( cluster_word_list[1], ' getPatternline(cluster_word_list)[0][1].replace(" ","")'))


[('Cluster Name', ' TMPCLST01')]


In [117]:
#@title テキストの分割を実行

# 分割するテキストフォルダを初期化
initConfigfolder()

# 元テキストから不要なテキストを削除
replaceConfig(  )

# 元テキストを分割
splitConfig( )


[' #ã\x83\x91ã\x83©ã\x83¡ã\x83¼ã\x82¿ã\x82·ã\x83¼ã\x83\x88ä½\x9cæ\x88\x90ç\x94¨ã\x81«å¿\x85è¦\x81ã\x81ªã\x83\x91ã\x83©ã\x83¡ã\x83¼ã\x82¿ã\x81®ã\x81¿æ\x8a½å\x87º\n\n #--\n\n ',
 ' setup ä¸\x80æ\x8b¬è¡¨ç¤º\n set -rows 0\n\n set -privilege advanced\n\nWarning: These advanced commands are potentially dangerous; use them only when directed to do so by NetApp personnel.\nDo you want to continue? {y|n}: y\n\n ',
 ' Node \n\n node_infomation\n node         model  serialnumber  \nTMPCLST01-01 SIMBOX 4034389-06-2\nTMPCLST01-02 SIMBOX 4082368-50-7\n\n\n\n ONTAP_version\n  node         version \nTMPCLST01-01 9.10.1: Sat Jan 15 10:49:44 UTC 2022\nTMPCLST01-02 9.10.1: Sat Jan 15 10:49:44 UTC 2022\n\n\n\n license\n serial-number package owner  \n1-80-000011   Base    TMPCLST01\n1-80-000011   SnapMirror_Sync TMPCLST01\n1-81-0000000000000004034389062 NFS TMPCLST01-01\n1-81-0000000000000004034389062 CIFS TMPCLST01-01\n1-81-0000000000000004034389062 iSCSI TMPCLST01-01\n1-81-0000000000000004034389062 FCP 

In [118]:
#@title エクセルファイルへの書き込み
import openpyxl

# ... (他のコード) ...

# エクセルファイルのワークブックを作成
book = openpyxl.Workbook()

#↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
# 余計なシートを削除--------------------------------

# 表紙, 設置先情報, 改訂履歴シートを作成する
createCover(cluster_name)
createCustomer()
createHistory()

# コンフィグシートを作成し、テーブルを作成する
createConfigsheet()
createTable()

# シート名を変更する
changeSheetname()

# 既存のシート名を取得
sheet_names = book.sheetnames

# 削除対象のシート名リスト
remove_sheets = ['Sheet']  # 'Sheet' のみ削除

# 存在するシートのみ削除
for sheet_name in remove_sheets:
  if sheet_name in sheet_names:
    book.remove(book[sheet_name])

# エクセルファイルを保存-----------------------------
book.save('/content/01_設定シート_'+ cluster_name +'.xlsx')

# ... (他のコード) ...








In [119]:
#@title ファイルダウンロード
files.download('/content/01_設定シート_'+ cluster_name +'.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>